In [ ]:
pip install deepface flask pillow flask-cors numpy opencv-python


In [2]:
!pip install flask pyngrok


In [3]:
!ngrok authtoken 2uupJNTPLof8UaeQ8aTj8gWAfBY_2PVV6ciL14KWDQqcrxVVh

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from deepface import DeepFace
import os
from PIL import Image
from io import BytesIO
import numpy as np

app = Flask(__name__)

# Function to convert numpy types to regular Python types
def convert_numpy_types(data):
    if isinstance(data, np.generic):
        return data.item()  # Convert numpy types to native Python types
    elif isinstance(data, dict):
        return {key: convert_numpy_types(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_numpy_types(item) for item in data]
    return data

@app.route('/detect_emotion', methods=['POST'])
def detect_emotion():
    try:
        # Retrieve the image from the request
        file = request.files['image']

        # Open the image using PIL to ensure it's in a valid format
        img = Image.open(file.stream)

        # Save the image to a temporary path as a JPEG to ensure compatibility
        temp_image_path = "/tmp/temp_image.jpg"
        img = img.convert("RGB")  # Ensure it's in RGB format
        img.save(temp_image_path)

        # Run DeepFace to analyze the image and predict emotion
        analysis = DeepFace.analyze(temp_image_path, actions=['emotion'])

        # Convert the analysis results to a JSON serializable format
        analysis = convert_numpy_types(analysis)

        # Return the analysis result
        return jsonify({
            'dominant_emotion': analysis[0]['dominant_emotion'],
            'emotion_details': analysis[0]['emotion']
        })
    except Exception as e:
        return jsonify({'error': str(e)})

# Expose the Flask app using ngrok
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run(port=5000)


25-04-17 19:55:14 - Directory /root/.deepface has been created
25-04-17 19:55:14 - Directory /root/.deepface/weights has been created
Public URL: NgrokTunnel: "https://4a03-34-125-50-184.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 19:59:23] "GET /detect_emotion HTTP/1.1" 405 -


25-04-17 19:59:32 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 112MB/s]
INFO:werkzeug:127.0.0.1 - - [17/Apr/2025 19:59:34] "POST /detect_emotion HTTP/1.1" 200 -
